In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

In [11]:
droot = "/glade/p/eol/stephens/longcoll/apo-forward-data/TM5-NOAA"
path = f'{droot}/molefrac_glb100x100_2020.nc'
print(path)
ds = xr.open_dataset(path)
ds

/glade/p/eol/stephens/longcoll/apo-forward-data/TM5-NOAA/molefrac_glb100x100_2020.nc


<xarray.Dataset>
Dimensions:              (time: 366, calendar_components: 6, longitude: 360, latitude: 180, level: 137, boundary: 138)
Coordinates:
  * time                 (time) datetime64[ns] 2020-01-01T12:00:00 ... 2020-1...
  * calendar_components  (calendar_components) int32 1 2 3 4 5 6
  * longitude            (longitude) float64 -180.0 -179.0 ... 178.0 179.0
  * latitude             (latitude) float64 -90.0 -89.0 -88.0 ... 87.0 88.0 89.0
  * level                (level) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137
  * boundary             (boundary) int32 1 2 3 4 5 6 ... 134 135 136 137 138
Data variables: (12/22)
    decimal_date         (time) float64 ...
    time_components      (time, calendar_components) int32 ...
    apo_jena             (time, level, latitude, longitude) float32 ...
    o2_cesm              (time, level, latitude, longitude) float32 ...
    co2_cesm             (time, level, latitude, longitude) float32 ...
    n2_cesm              (time, level, latitude, longitude) float32 ...
    ...                   ...
    blh                  (time, latitude, longitude) float32 ...
    temperature          (time, level, latitude, longitude) float32 ...
    specific_humidity    (time, level, latitude, longitude) float32 ...
    air_mass             (time, level, latitude, longitude) float32 ...
    u                    (time, level, latitude, longitude) float32 ...
    v                    (time, level, latitude, longitude) float32 ...
Attributes:
    notes:                  This file contains CarbonTracker mole fractions a...
    disclaimer:             CarbonTracker is an open product of the NOAA Eart...
    email:                  carbontracker.team@noaa.gov
    url:                    https://carbontracker.noaa.gov
    institution:            NOAA Global Monitoring Laboratory
    conventions:            CF-1.5
    history:                Tue Jul 19 22:33:17 2022: ncrcat -o /work2/noaa/c...
    source:                 CarbonTracker
    NCO:                    netCDF Operators version 4.9.3 (Homepage = http:/...
    nco_input_file_number:  366
    nco_input_file_list:    ./20191222_000000__20200119_000000/molefrac_glb10...

In [ ]:
ds.pressure.sel(longitude=45, latitude=45, method='nearest').plot()

In [ ]:
sno_vars = [
    'apo_jena', 
    'co2_cesm', 
    'co2_gridfed', 
    'co2_oco2mip', 
    'co2_somffn', 
    'n2_cesm', 
    'n2_era', 
    'o2_cesm', 
    'o2_gk01r16', 
    'o2_gridfed',
]
for v in ['pressure'] + sno_vars + ['co2bg']:
    
    if v not in ds:
        print(f'missing {v}')
        continue
    
    if v in sno_vars:            
        assert ds[v].units == 'micromol mol-1'
    
    print(v)
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))        
       
    if 'level' in ds[v].dims:
        isel = dict(level=0)
    elif 'boundary' in ds[v].dims:
        isel = dict(boundary=0)

    with xr.set_options(keep_attrs=True):    
        surface = ds[v].where(np.abs(ds[v]) < 1.0e30).isel(**isel)
        zonal_mean = ds[v].where(np.abs(ds[v]) < 1.0e30).mean('longitude')
    
    if ds[v].units == 'micromol mol-1':
        surface *= 1e6
        zonal_mean *= 1e6
        
    vmin, vmax = np.percentile(surface.values, [2.5, 97.5])
    
    surface.plot(ax=axs[0]) #, vmin=vmin, vmax=vmax)    
    print(f'surface: range = {surface.min().values:0.2f}, {surface.max().values:0.2f}')

    vmin, vmax = np.percentile(zonal_mean.values, [2.5, 97.5])
    zonal_mean.plot(ax=axs[1]) #, vmin=vmin, vmax=vmax)
    print(f'zonal mean: range = {zonal_mean.min().values:0.2f}, {zonal_mean.max().values:0.2f}')
    
    plt.suptitle(v, fontweight='bold')
    plt.tight_layout();
    print()

## Interpolate model to ObsPack locations


In [31]:
obspack_dir = "/glade/work/mclong/sno-analysis/obspack_co2_1_GLOBALVIEWplus_v7.0_2021-08-18/data/nc"
surface_files = [
    f"{obspack_dir}/co2_alt_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_brw_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_cba_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_cgo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_kum_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_ljo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_mlo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_psa_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_smo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_spo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_gould_shipboard-insitu_1_allvalid.nc",
]

stns = []
dsets_stn = {}
for f in surface_files:
    stn = os.path.basename(f).split('_')[1]
    stns.append(stn)
    with xr.open_dataset(f) as dsi:
        dsets_stn[stn] = dsi.load() 

In [42]:
import esmf_tools
from functools import partial

interp_points = partial(
    esmf_tools.esmf_interp_points,
    ds_in=ds[sno_vars].isel(level=-1),
    lon_field_name='longitude',
    lat_field_name='latitude',
)

gould_lat_bin_e = np.arange(-66, -54, 2)
gould_lat_bin_c = np.vstack((gould_lat_bin_e[:-1], gould_lat_bin_e[1:])).mean(axis=0)


dsets_model = {}
for key, dsi in dsets_stn.items():
    if key == 'gould':        
        ndx = np.where(dsi.latitude < -40)[0]
        dsii = dsi.isel(obs=ndx)
        dsii = dsii.groupby_bins(dsii.latitude, gould_lat_bin_e).mean()
        for i, (lon, lat) in enumerate(zip(dsii.longitude, dsii.latitude)):
            dsets_model[f'{key}_{np.abs(gould_lat_bin_c[i]):0.0f}S'] = interp_points(
                locs_lon=lon.expand_dims('loc'), 
                locs_lat=lat.expand_dims('loc'),
            ).isel(loc=0, drop=True)
    else:
        assert dsi.latitude.std() < 0.01 and dsi.longitude.std() < 0.01, key
        dsets_model[key] = interp_points(
            locs_lon=dsi.longitude.mean().expand_dims('loc'), 
            locs_lat=dsi.latitude.mean().expand_dims('loc'),
        ).isel(loc=0, drop=True)

       